# Initialize

In [1]:
!pip install -U spacy

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

# Regulation from text service api

In [6]:
text_service_subsections = [
                                {
                                    "content": "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.",
                                    "uid": "115574bc-b56d-4908-406c-08d8c109f9ce",
                                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/1"
                                },
                                {
                                    "content": "Når det er personer om bord skal brannslukkingsanlegget bare kunne utløses manuelt.",
                                    "uid": "52b3b082-0808-4240-406d-08d8c109f9ce",
                                    "url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/2"
                                }
                            ]

In [8]:
for test in text_service_subsections:
    print(test)

{'content': '(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.', 'uid': '115574bc-b56d-4908-406c-08d8c109f9ce', 'url': 'http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/1'}
{'content': 'Når det er personer om bord skal brannslukkingsanlegget bare kunne utløses manuelt.', 'uid': '52b3b082-0808-4240-406d-08d8c109f9ce', 'url': 'http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/2'}


In [11]:
url = "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/15/subsections/1/sentences/2"

url_split_in_list = url.split('/')

for number, part in enumerate(url_split_in_list):
    print(str(number) + " " + part)

0 http:
1 
2 sdir-d-aks-core.norwayeast.cloudapp.azure.com
3 v1
4 regulations
5 2013
6 11
7 22
8 1404
9 chapters
10 3
11 sections
12 15
13 subsections
14 1
15 sentences
16 2


In [12]:
#
# Text service metadata
#

api_domain = url_split_in_list[2] # sdir-d-aks-core.norwayeast.cloudapp.azure.com
api_version = url_split_in_list[3] # v1

regularion_variable_name = url_split_in_list[4] # regulations
regularion_year = url_split_in_list[5] # 2013
regularion_month = url_split_in_list[6] # 11
regularion_day = url_split_in_list[7] # 22
regularion_id = url_split_in_list[8] # 1404

chapter_variable_name = url_split_in_list[9] # chapters
chapter_number = url_split_in_list[10] # 3

section_variable_name = url_split_in_list[11] # sections
section_number = url_split_in_list[12] # 15

part_variable_name = url_split_in_list[13] # subsections
part_number = url_split_in_list[14] # 1

sub_part_variable_name = url_split_in_list[15] # sentences
sub_part_number = url_split_in_list[16] # 2

# Code

In [4]:
import spacy
#from spacy.lang.en import English
from spacy.lang.nb import Norwegian
from spacy.matcher import Matcher

# Blank model Norwegian().
# Does not have a POS tagger.
nlp = Norwegian()

text = "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg."

doc = nlp(text)

matcher = Matcher(nlp.vocab)

length_overall_pattern = [{"TEXT": {"REGEX": "[0-9]+[,]+[0-9]|[0-9]"}},
{"LOWER": {"IN": ["og"]}},
{"TEXT": {"REGEX": "[0-9]+[,]+[0-9]|[0-9]"}},
{"LOWER": {"IN": ["meter"]}}]

matcher.add("LENGTH_OVERALL", None, length_overall_pattern)

result = []

for match_id, token_start, token_end in matcher(doc):

    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    span = doc[token_start:token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    match_id_as_string = nlp.vocab.strings[match_id]

    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

#result

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"LENGTH_OVERALL": "yellow"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [5]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import identify_length_overall_in_text

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

identify_length_overall_in_text("(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.")

[{'start': 37, 'end': 54, 'label': 'LENGTH_OVERALL'}]

In [14]:
forward_lines_as_spacy_ner_format = [
  {'text': "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.", 'ents': [], 'title': 0},
  {'text': '\n', 'ents': [], 'title': 1},
  {'text': 'På fartøy med største lengde 8 til 15 meter skal ovnen kontrolleres hos godkjent foretak innen seks måneder etter at den er installert.', 'ents': [], 'title': 2}]

In [16]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import identify_length_overall_in_text

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

for line in forward_lines_as_spacy_ner_format:

    text = line['text']

    if text != "\n":
        result_in_a_list = identify_length_overall_in_text(text)

        if result_in_a_list:
            print(result_in_a_list)

[{'start': 37, 'end': 54, 'label': 'LENGTH_OVERALL'}]
[{'start': 29, 'end': 43, 'label': 'LENGTH_OVERALL'}]


# Merge spacy result with spacy ner formated list of text

In [17]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import merge_spacy_entity_results_to_spacy_ner_format

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

spacy_ner_formated_text_line = {'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n',
  'ents': [],
  'title': 53}

spacy_ner_entities_to_be_merged_in_as_a_list = [{'start': 16, 'end': 19, 'label': 'DATE'}, {'start': 6, 'end': 9, 'label': 'DATE'}]

print(merge_spacy_entity_results_to_spacy_ner_format(spacy_ner_formated_text_line,spacy_ner_entities_to_be_merged_in_as_a_list))

spacy_ner_formated_text_line = {'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n',
  'ents': [{'start': 16, 'end': 19, 'label': 'DATE'}, {'start': 6, 'end': 9, 'label': 'DATE'}],
  'title': 53}

print("")

print(merge_spacy_entity_results_to_spacy_ner_format(spacy_ner_formated_text_line,spacy_ner_entities_to_be_merged_in_as_a_list))

{'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n', 'ents': [{'start': 6, 'end': 9, 'label': 'DATE'}, {'start': 16, 'end': 19, 'label': 'DATE'}], 'title': 53}

{'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material 

# identify_length_overall_in_text

In [18]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import identify_length_overall_in_spacy_lines

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

result = identify_length_overall_in_spacy_lines(forward_lines_as_spacy_ner_format)
#print(result)

options = {"colors": {"LENGTH_OVERALL": "yellow"}}

# display result with spacy styling
from spacy import displacy
displacy.render(result, style='ent', manual=True, options=options)